# Modules

In [32]:
import pyspark
from pyspark.sql.functions import *

from pyspark.sql.types import *

# Constantes

In [33]:
PATH_DATA = r"C:\Users\ahoun\Documents\Documents\Nuku Data\Real-estate-prices-etl\Data"
PATH_DATA_RAW = rf"{PATH_DATA}\Raw"
PATH_DATA_TRANSFORMED = rf"{PATH_DATA}\Clean"

# Processing

In [34]:
# Initialisez une session Spark
spark = pyspark.sql.SparkSession.builder \
    .appName("Realstate") \
    .getOrCreate()

In [ ]:
spark.active()

In [ ]:
# Read JSON file into dataframe
df = spark.read.json(r'C:\Users\ahoun\Documents\Documents\Nuku Data\Real-estate-prices-etl\Data\Raw\Page_2\data_page_2.json')
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- room: string (nullable = true)
 |    |    |-- surface: string (nullable = true)
 |    |    |-- title: string (nullable = true)



In [ ]:
data = spark.read.option("multiline","true").json(r'C:\Users\ahoun\Documents\Documents\Nuku Data\Real-estate-prices-etl\Data\Raw\Page_2\data_page_2.json')
data.show()

+--------------------+
|                data|
+--------------------+
|[{Situé au centre...|
+--------------------+



In [ ]:
# df = spark.read.format('org.apache.spark.sql.json') \
#         .load(r'C:\Users\ahoun\Documents\Documents\Nuku Data\Real-estate-prices-etl\Data\Raw\Page_2\data_page_2.json')
# df.show()

In [ ]:
# # Initialisez une session Spark
# # spark = SparkSession.builder.appName("Lecture_JSON").getOrCreate()

# # Définissez le schéma au format JSON
# custom_schema = StructType([
#     StructField("data", ArrayType(StructType([
#         StructField("description", StringType(), nullable=True),
#         StructField("id", LongType(), nullable=True),
#         StructField("price", StringType(), nullable=True),
#         StructField("room", StringType(), nullable=True),
#         StructField("surface", StringType(), nullable=True),
#         StructField("title", StringType(), nullable=True)
#     ]),True), nullable=True)
# ])
                           
# # Lisez le fichier JSON en utilisant le schéma JSON
# df = spark.read.json(r'C:\Users\ahoun\Documents\Documents\Nuku Data\Real-estate-prices-etl\Data\Raw\Page_2\data_page_2.json', 
#           schema=custom_schema, multiLine=True)

# # Affichez le DataFrame
# df.show()

In [ ]:
df.show()

+--------------------+
|                data|
+--------------------+
|[{Situé au centre...|
+--------------------+



In [ ]:
# spark.stop()
# # Initialisez une session Spark
# spark = pyspark.sql.SparkSession.builder \
#     .appName("Realstate") \
#     .getOrCreate()

# df = spark.read.json(r'C:\Users\ahoun\Documents\Documents\Nuku Data\Real-estate-prices-etl\Data\Raw\Page_2\data_page_2.json')

In [ ]:
df2 = df.withColumn("id", explode(col("data.id"))) \
    .withColumn("title", explode(col("data.title"))) \
    .withColumn("description", explode(col("data.description"))) \
    .withColumn("room", explode(col("data.room"))) \
    .withColumn("price", explode(col("data.price"))) \
    .withColumn("surface", explode(col("data.surface"))) \
    .select('id', 'title', 'description', 'room', 'price', 'surface')

df2.show()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "c:\Users\ahoun\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ahoun\AppData\Local\Temp\ipykernel_3264\2855427701.py", line 9, in <module>
    df2.show()
  File "c:\Users\ahoun\anaconda3\lib\site-packages\pyspark\sql\dataframe.py", line 959, in show
    print(self._jdf.showString(n, 20, vertical))
  File "c:\Users\ahoun\anaconda3\lib\site-packages\py4j\java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "c:\Users\ahoun\anaconda3\lib\site-packages\pyspark\errors\exceptions\captured.py", line 179, in deco
    return f(*a, **kw)
  File "c:\Users\ahoun\anaconda3\lib\site-packages\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another e

ConnectionRefusedError: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée

In [ ]:
# df2 = df2.withColumn("title",explode(col("data.title")))
#         # .select('id', 'title', 'description', 'room', 'price', 'surface')
df2.printSchema()
df2.show(2)

In [ ]:
df.select(explode('data.id').alias('id'),
            explode('data.title').alias('title'),
            explode('data.description').alias('description'),
            explode('data.room').alias('room'),
            explode('data.surface').alias('surface'),
            explode('data.price').alias('price')).show()

AnalysisException: [UNSUPPORTED_GENERATOR.MULTI_GENERATOR] The generator is not supported: only one generator allowed per SELECT clause but found 6: "explode(data.id)", "explode(data.title)", "explode(data.description)", "explode(data.room)", "explode(data.surface)", "explode(data.price)".

In [ ]:
df = df.select(
    explode('data.id').alias('id'),
    explode('data.title').alias('title'),
    explode('data.description').alias('description'),
    explode('data.room').alias('room'),
    explode('data.surface').alias('surface'),
    explode('data.price').alias('price')
)

df.show()

AnalysisException: [UNSUPPORTED_GENERATOR.MULTI_GENERATOR] The generator is not supported: only one generator allowed per SELECT clause but found 6: "explode(data.id)", "explode(data.title)", "explode(data.description)", "explode(data.room)", "explode(data.surface)", "explode(data.price)".

In [ ]:
data.select('data', explode('data')).show()